## Select observations and data products from MAST
#### and load data products into `jdaviz` and `mast_aladin_lite`

You'll need roman-specific dependencies for jdaviz, and the latest astroquery:

In [ ]:
# pip install git+https://github.com/astropy/astroquery.git
# pip install jdaviz[roman]

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord

from astroquery.mast import MastMissions
from astropy.table import Table
from mast_aladin_lite import MastTable, AppSidecar


target = SkyCoord(ra=270, dec=66, unit='deg')
mast = MastMissions(mission='roman')
mast._service_api_connection.SERVICE_URL = 'https://masttest.stsci.edu'
mast._service_api_connection.REQUEST_URL = 'https://masttest.stsci.edu/search/roman/api/v0.1/'
mast._service_api_connection.MISSIONS_DOWNLOAD_URL = 'https://masttest.stsci.edu/search/'

obs = mast.query_region(target, radius=0.3 * u.deg)

We can construct a `MastTable` widget with the results of the observation query:

In [ ]:
mast_table_obs = MastTable(obs)
mast_table_obs

Select the first observation in the table. You can do this by clicking the checkbox in the table widget, or with this call from the API:

In [ ]:
mast_table_obs.selected_rows = mast_table_obs.items[0:2]

The `MastTable` above contains file sets of observations, but not the files themselves for data products within those file sets. We run a second query to get the data products with `MastMissions.get_product_list`, and create a new `MastTable` widget for those results:

In [ ]:
products = mast.get_product_list(mast_table_obs.selected_rows_table)
cal_products = mast.filter_products(products, file_suffix='_cal', calibration_software_version='9.9.0')
mast_table_products = MastTable(cal_products, unique_column='uri')
mast_table_products

Select the first data product in the product list:

In [ ]:
mast_table_products.selected_rows = mast_table_products.items[:]

In [ ]:
download_path = mast.download_products(mast_table_products.selected_rows_table)['Local Path']

Pop open a sidecar with `jdaviz` and `mast-aladin-lite`:

In [ ]:
mast_aladin, viz = AppSidecar.open()

In [ ]:
from mast_aladin_lite import MastAladin

mast_aladin.target = f"{target.ra.deg} {target.dec.deg}"
mast_aladin.fov = 0.5

mast_aladin.add_graphic_overlay_from_stcs(mast_table_obs.selected_rows_table['s_region']);

In [ ]:
from jdaviz import Imviz

with viz.batch_load():
    for path in download_path:
        viz.load_data(path, gwcs_to_fits_sip=True)

# link the images by WCS:
viz.link_data('wcs')

# make multiple data layers visible at once:
plot_opts = viz.plugins['Plot Options']._obj

# using private API for now:
for layer in plot_opts.layer.choices:
    plot_opts.layer_selected = layer
    for lyr in plot_opts.layer.selected_obj:
        lyr.composite._allow_bad_alpha = True
        lyr.state.alpha = 0.99
        lyr.state.cmap.set_bad(color='k', alpha=0)
        lyr.force_update()

In [ ]:
## API equivalent of clicking the buttons that appear in the above widget:

# mast_table_products.vue_open_selected_rows_in_jdaviz();
# mast_table_products.vue_open_selected_rows_in_aladin();